In [1]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import torch
import torch.nn as nn
import torch
torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from soft_embedding import SoftEmbedding
from dataset import BiasDataset
import torchvision
from torch.utils.data import Dataset, DataLoader
import numpy as np
import math
import pandas as pd

In [2]:
torch.device('cuda' if torch.cuda.is_available() else 'cpu')

device(type='cuda')

In [3]:
### TO CHECK WHETHER THE output_1.logits[:, -1, :] returns the next word logits for each sentence in the batch.

def get_next_word_log_probabilities(model_input_1, model_input_2, model, logsoftmax):
    #import pdb; pdb.set_trace()
    output_1 = model(input_ids = model_input_1[0],attention_mask = model_input_1[1])     
    #here we get the logits from the output, the logits are obtained from the output.logits, here we first get
    #the [batch_size, sequence_len, vocab_size]
    #next_word_logits_1 = torch.stack([torch.index_select(seq_logits, 0, torch.tensor([i+1])) for i, seq_logits in enumerate(output_1.logits)])
    next_word_logits_1 = output_1.logits[:, -1, :]
    next_word_log_probs_1 = logsoftmax(next_word_logits_1)
    
    output_2 = model(input_ids = model_input_2[0],attention_mask = model_input_2[1] ) 
    #next_word_logits_2 = torch.stack([torch.index_select(seq_logits, 0, torch.tensor([i+1])) for i, seq_logits in enumerate(output_2.logits)])
    next_word_logits_2 = output_2.logits[:, -1, :]
    next_word_log_probs_2 = logsoftmax(next_word_logits_2)    

    return next_word_log_probs_1, next_word_log_probs_2


In [4]:
def get_model(n_tokens = 5 , initialize_from_vocab = True):
    model = GPT2LMHeadModel.from_pretrained('gpt2')
    #freezing all the layers, and only keeping the tokens one active

    for param in model.parameters():
        param.requires_grad_(False)

    s_wte = SoftEmbedding(model.get_input_embeddings(), 
                          n_tokens=n_tokens, 
                          initialize_from_vocab=initialize_from_vocab)
    model.set_input_embeddings(s_wte)
    return model

In [7]:
def run(model, dataloader, num_epochs = 5, train = True):
    ''' 
    if train is false, we are only testing the model, there will be only one loss value
    if train true:
            in this case, we update the soft embedding tokens and run for the desired number of epochs.
    '''
    logsoftmax = nn.LogSoftmax(dim = 0)
    kl_loss = nn.KLDivLoss(reduction="batchmean", log_target=True) #can try mean reduction
    if not train:
        loss = 0
        curr_loss = []
        for iter, data in enumerate(dataloader):
            group_1, group_2 = data[0], data[1]
            with torch.no_grad():
                log_probs1, log_probs2 = get_next_word_log_probabilities(data[0], data[1], model, logsoftmax)
                loss = torch.mean(torch.stack([kl_loss(log_probs1[i], log_probs2[i]) for i in range(len(log_probs1))]))
                curr_loss.append(loss)
            #print(f'loss at batch : {iter} is {loss.item()}')

        return torch.mean(torch.stack(curr_loss))
    
    else:
        optimizer = torch.optim.Adam(model.parameters(), lr=5e-2)

        epoch_loss = []
        
        for epoch in range(num_epochs):

            print(f'epoch : {epoch}')
            loss = 0
            curr_loss = []
            for iter, data in enumerate(dataloader):
                group_1, group_2 = data[0], data[1]

                log_probs1, log_probs2 = get_next_word_log_probabilities(data[0], data[1], model, logsoftmax)
                #import pdb; pdb.set_trace()
                #loss = kl_loss(log_probs1, log_probs2)
                loss = torch.mean(torch.stack([kl_loss(log_probs1[i], log_probs2[i]) for i in range(len(log_probs1))]))
                optimizer.zero_grad()

                loss.backward()
                optimizer.step()
                curr_loss.append(loss)
                if iter%5==0:
                    print(f'loss at batch : {iter} is {loss.item()}')

            epoch_loss.append(torch.mean(torch.stack(curr_loss)))
        return epoch_loss
        
        

# train on gender and test the performance on race

In [6]:
train_dataset = BiasDataset("../data/train_gender.csv", n_tokens = 5)
train_dataloader = train_dataset.get_dataloader(batch_size=64)

gender_dataset = BiasDataset("../data/test_gender.csv", n_tokens = 5)
gender_dataloader = gender_dataset.get_dataloader(batch_size=64)

race_dataset = BiasDataset("../data/test_race.csv", n_tokens = 5)
race_dataloader = race_dataset.get_dataloader(batch_size=64)

len(train_dataset), len(gender_dataset), len(race_dataset)

(5432, 1358, 2716)

In [ ]:
model = get_model(n_tokens = 5 , initialize_from_vocab = True)

'''first we are training the model, each case pass a different dataloader corresponding to a different dataset'''

gender_loss_before_training = run(model, gender_dataloader, 2, False)

print(f'gender_loss_before_training is {gender_loss_before_training}')
race_loss_before_training = run(model, race_dataloader, 2, False)
print(f'race_loss_before_training is {race_loss_before_training}')


epoch_loss_training = run(model, train_dataloader, 5, True)
print(f'epoch loss is {epoch_loss_training}')

gender_loss_after_training = run(model, gender_dataloader, 2, False)

race_loss_after_training = run(model, race_dataloader, 2, False)

gender_loss_before_training is 0.011328664608299732
race_loss_before_training is 0.002282505389302969
epoch : 0
loss at batch : 0 is 0.0021197027526795864
loss at batch : 5 is 0.004379349760711193
loss at batch : 10 is 0.008802814409136772
loss at batch : 15 is 0.003076541004702449


In [ ]:
gender_loss_before_training, race_loss_before_training, epoch_loss_training, gender_loss_after_training, race_loss_after_training


# train on race and test the performance on gender

In [ ]:
train_dataset = BiasDataset("../data/train_race.csv", n_tokens = 5)
train_dataloader = train_dataset.get_dataloader(batch_size=64)

gender_dataset = BiasDataset("../data/test_gender.csv", n_tokens = 5)
gender_dataloader = gender_dataset.get_dataloader(batch_size=64)

race_dataset = BiasDataset("../data/test_race.csv", n_tokens = 5)
race_dataloader = race_dataset.get_dataloader(batch_size=64)


model = get_model(n_tokens = 5 , initialize_from_vocab = True)

'''
first we are training the model, each case pass a different dataloader corresponding to a different dataset,

'''

gender_loss_before_training = run(model, gender_dataloader, 2, False)
race_loss_before_training = run(model, race_dataloader, 2, False)


epoch_loss_training = run(model, train_dataloader, 5, True)


gender_loss_after_training = run(model, gender_dataloader, 2, False)
race_loss_after_training = run(model, race_dataloader, 2, False)

In [ ]:
## initial loss - 


In [ ]:
loss_before_training

In [ ]:
epoch_loss_training

In [ ]:
loss_after_training